In [1]:
# Cài đặt các thư viện cốt lõi cho RAG và LLM
!pip install -q \
    torch \
    transformers \
    accelerate \
    bitsandbytes \
    llama-index \
    llama-index-readers-file \
    llama-index-llms-huggingface \
    llama-index-embeddings-huggingface \
    # Thư viện cho Jupyter/Colab để hiển thị tiến trình


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 70.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.3/303.3 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.9/323.9 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.4/144.4 kB 6.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the sour

In [2]:
# Cài đặt các thư viện cốt lõi cho RAG và LLM
!pip install -q \
    torch \
    transformers \
    accelerate \
    bitsandbytes \
    llama-index \
    llama-index-readers-file \
    llama-index-llms-huggingface \
    llama-index-embeddings-huggingface \
    # Thư viện cho Jupyter/Colab để hiển thị tiến trình


In [3]:
!pip install pypdf
!pip install -q tqdm

In [4]:
!hf auth login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Y
Token is valid (permission: read).
The token `test` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pus

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
import os, shutil, torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

from llama_index.core import (
    Settings,
    VectorStoreIndex,
    load_index_from_storage,
    StorageContext,
)
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core.ingestion.cache import IngestionCache
from llama_index.core.readers import SimpleDirectoryReader
from llama_index.core.node_parser import TokenTextSplitter
from llama_index.core.extractors import SummaryExtractor, TitleExtractor
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.huggingface import HuggingFaceLLM


In [16]:
# --- ĐỊNH NGHĨA CÁC BIẾN TOÀN CỤC ---
DOCX_PATH = "/content/drive/MyDrive/DACNTT/DMS-5.pdf"
CACHE_FILE = "/content/drive/MyDrive/DACNTT/data/cache/pipeline_cache.json"
INDEX_STORAGE = "/content/drive/MyDrive/DACNTT/data/index_store"

os.makedirs(os.path.dirname(CACHE_FILE), exist_ok=True)
os.makedirs(INDEX_STORAGE, exist_ok=True)
print(f"✅ (Bước 1) Các đường dẫn đã được định nghĩa. DOCX_PATH: {DOCX_PATH}")
# --- KẾT THÚC ĐỊNH NGHĨA BIẾN ---



✅ (Bước 1) Các đường dẫn đã được định nghĩa. DOCX_PATH: /content/drive/MyDrive/DACNTT/DMS-5.pdf


In [17]:

def setup_llm_and_embedding():
    global embed_model, llm

    # ---- Embedding model ----
    EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
    print(f"⏳ (Bước 1) Đang tải Embedding model: {EMBEDDING_MODEL}")
    embed_model = HuggingFaceEmbedding(model_name=EMBEDDING_MODEL)
    Settings.embed_model = embed_model
    print("✅ (Bước 1) Embedding Model sẵn sàng.\n")

    # ---- LLM model (Llama 3.1 - 8B 4bit) ----
    LLM_MODEL_NAME = "/content/drive/MyDrive/llama3-8b-mental-health-merged"
    print(f"⏳ (Bước 1) Đang tải LLM 4-bit: {LLM_MODEL_NAME}")

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.float16
    )

    tokenizer = AutoTokenizer.from_pretrained(LLM_MODEL_NAME, trust_remote_code=True)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    # --- SỬA LỖI 1 (Ổ khóa & Chìa khóa) ---
    print("ℹ️ (Bước 1) Đang gán 'chat_template'...")
    tokenizer.chat_template = (
        "<|begin_of_text|>"
        "{% for message in messages %}"
            "{% if message['role'] == 'user' %}"
                "<|start_header_id|>user<|end_header_id|>\n\n{{ message['content'] }}<|eot_id|>" # FIXED: Removed the extra dot
            "{% elif message['role'] == 'system' %}"
                "<|start_header_id|>system<|end_header_id|>\n\n{{ message['content'] }}<|eot_id|>"
            "{% elif message['role'] == 'assistant' %}"
                "<|start_header_id|>assistant<|end_header_id|>\n\n{{ message['content'] }}<|eot_id|>"
            "{% endif %}"
        "{% endfor %}"
    )
    print("✅ (Bước 1) Đã gán 'chat_template' vào 'tokenizer' thành công.")
    # --- KẾT THÚC SỬA LỖI 1 ---

    model = AutoModelForCausalLM.from_pretrained(
        LLM_MODEL_NAME,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True
    )

    # --- SỬA LỖI 2 (Lỗi Lặp - Looping Error) ---
    # Lấy ID của các token dừng (stop tokens)
    eos_token_id = tokenizer.eos_token_id
    # Llama 3 dùng <|eot_id|> (End of Turn) làm tín hiệu dừng
    eot_token_id = tokenizer.convert_tokens_to_ids("<|eot_id|>")

    print(f"ℹ️ (Bước 1) Đang gán 'stopping_ids' (ID: {eos_token_id} và {eot_token_id})...")
    # --- KẾT THÚC SỬA LỖI 2 ---

    llm = HuggingFaceLLM(
        model=model,
        tokenizer=tokenizer, # <-- Đưa tokenizer đã sửa lỗi 1 vào đây
        context_window=8192,
        max_new_tokens=512,
        generate_kwargs={
            "temperature": 0.3,
            "pad_token_id": eos_token_id
        },
        # --- THÊM THAM SỐ SỬA LỖI 2 VÀO ĐÂY ---
        stopping_ids=[eos_token_id, eot_token_id]
    )

    Settings.llm = llm
    print("✅ (Bước 1) Llama 3.1-8B 4bit đã sẵn sàng (Đã sửa lỗi lặp).\n")

# Chạy setup ngay lập tức để khởi tạo embed_model và llm
setup_llm_and_embedding()

⏳ (Bước 1) Đang tải Embedding model: sentence-transformers/all-MiniLM-L6-v2
✅ (Bước 1) Embedding Model sẵn sàng.

⏳ (Bước 1) Đang tải LLM 4-bit: /content/drive/MyDrive/llama3-8b-mental-health-merged
ℹ️ (Bước 1) Đang gán 'chat_template'...
✅ (Bước 1) Đã gán 'chat_template' vào 'tokenizer' thành công.


/usr/local/lib/python3.12/dist-packages/transformers/quantizers/auto.py:239: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

ℹ️ (Bước 1) Đang gán 'stopping_ids' (ID: 128009 và 128009)...
✅ (Bước 1) Llama 3.1-8B 4bit đã sẵn sàng (Đã sửa lỗi lặp).



In [18]:
def ingest_documents():
    print("📂 (Bước 2) Đọc tài liệu ...")

    if not os.path.exists(DOCX_PATH):
        print(f"❌ LỖI: File không tồn tại tại: {DOCX_PATH}")
        return []

    documents = SimpleDirectoryReader(input_files=[DOCX_PATH], filename_as_id=True).load_data()
    print(f"✅ (Bước 2) Đã tải {len(documents)} tài liệu.")

    if not documents:
        print("⚠️ (Bước 2) Không có tài liệu.")
        return []

    # ---- DÙNG CACHE (RẤT QUAN TRỌNG) ----
    try:
        cached_hashes = IngestionCache.from_persist_path(CACHE_FILE)
        print("🧠 (Bước 2) Dùng lại cache đã lưu.\n")
    except:
        cached_hashes = None
        print("ℹ️ (Bước 2) Không có cache, chạy mới.\n")
    # ---

    # --- PIPELINE NÂNG CAO (CHẬM) ---
    print("🚀 (Bước 2) Đang sử dụng pipeline NÂNG CAO với SummaryExtractor (CHẬM 5-8 TIẾNG)...")
    pipeline = IngestionPipeline(
        transformations=[
            TokenTextSplitter(chunk_size=512, chunk_overlap=20),


            #SummaryExtractor(summaries=["self"], prompt_template=CUSTOM_SUMMARY_EXTRACT_TEMPLATE),

            embed_model # Embed model phải chạy sau extractor
        ],
        cache=cached_hashes, # RẤT QUAN TRỌNG ĐỂ DÙNG CACHE
    )
    # --- KẾT THÚC PIPELINE ---

    nodes = pipeline.run(documents=documents)
    print(f"✅ (Bước 2) Đã tạo {len(nodes)} nodes (và tóm tắt) + tính embedding.")

    # Lưu cache sau khi chạy
    pipeline.cache.persist(CACHE_FILE)
    print(f"✅ (Bước 2) Đã lưu cache vào {CACHE_FILE}")
    return nodes

In [19]:
# =================================================================
# HÀM BUILD INDEX
# =================================================================
def build_indexes(nodes):
    print("⏳ Đang kiểm tra/tạo Index...")
    print(f"⚠️ Không tìm thấy Index. Tiến hành tạo Index mới.")

    if not nodes:
        raise ValueError("Không có nodes đầu vào để tạo Index.")

    storage_context = StorageContext.from_defaults()
    vector_index = VectorStoreIndex(nodes, storage_context=storage_context)
    vector_index.set_index_id("vector")

    print(f"⏳ Đang lưu trữ Index mới vào thư mục: {INDEX_STORAGE}...")
    storage_context.persist(persist_dir=INDEX_STORAGE)
    print("✅ Index mới đã được TẠO và LƯU TRỮ thành công.")

    return vector_index


In [20]:
# HÀM MAIN (CHẠY NGAY)
# =================================================================
def main():
    print("\n" + "="*50)
    print("🔹 BẮT ĐẦU CHẠY QUY TRÌNH BUILD DATA...")
    print("="*50)

    try:
        # 1️⃣ Ingest
        nodes = ingest_documents()
        print(f"✅ Ingestion hoàn tất. Đã tạo {len(nodes)} nodes.")

        # 2️⃣ Build index
        if nodes:
            index = build_indexes(nodes)
            print("✅ Hoàn tất xây dựng VectorStoreIndex.")
            return index
        else:
            print("⚠️ Không có nodes nào được tạo, bỏ qua bước Indexing.")
            return None
    except Exception as e:
        print(f"❌ LỖI NGHIÊM TRỌNG TRONG HÀM MAIN: {e}")
        print("👉 Hãy kiểm tra lại các bước 'setup' và đường dẫn file.")
        return None

# --- CHẠY MAIN NGAY LẬP TỨC ---
if 'embed_model' in globals(): # Chỉ chạy nếu Bước 1 đã thành công
    vector_index = main()
else:
    print("❌ BỎ QUA Bước 2: Vui lòng chạy Bước 1 (Setup) trước.")



🔹 BẮT ĐẦU CHẠY QUY TRÌNH BUILD DATA...
📂 (Bước 2) Đọc tài liệu ...
✅ (Bước 2) Đã tải 106 tài liệu.
🧠 (Bước 2) Dùng lại cache đã lưu.

🚀 (Bước 2) Đang sử dụng pipeline NÂNG CAO với SummaryExtractor (CHẬM 5-8 TIẾNG)...
✅ (Bước 2) Đã tạo 293 nodes (và tóm tắt) + tính embedding.
✅ (Bước 2) Đã lưu cache vào /content/drive/MyDrive/DACNTT/data/cache/pipeline_cache.json
✅ Ingestion hoàn tất. Đã tạo 293 nodes.
⏳ Đang kiểm tra/tạo Index...
⚠️ Không tìm thấy Index. Tiến hành tạo Index mới.
⏳ Đang lưu trữ Index mới vào thư mục: /content/drive/MyDrive/DACNTT/data/index_store...
✅ Index mới đã được TẠO và LƯU TRỮ thành công.
✅ Hoàn tất xây dựng VectorStoreIndex.


In [21]:

!pip install -q streamlit pyngrok

In [44]:
import gradio as gr
import traceback
from llama_index.core import Settings, VectorStoreIndex
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.core.prompts import PromptTemplate

# ========================
# 1️⃣ TEMPLATE DSM-5
# ========================
RAG_PROMPT_TEMPLATE = """\
Bạn là một **chuyên gia tâm lý AI** được phát triển bởi **AI VIETNAM**.
Nhiệm vụ của bạn là **trò chuyện, theo dõi và tư vấn** cho người dùng về **sức khỏe tâm thần hàng ngày**.

Thông tin người dùng (nếu có):
{user_info}

---

🎯 **Hướng dẫn hành vi của bạn trong cuộc trò chuyện:**

{context}

Người dùng hỏi: {user_message}
Trả lời:
"""

# ========================
# 2️⃣ LOAD PIPELINE & INDEX
# ========================
try:
    from __main__ import vector_index, llm, embed_model
except ImportError:
    print("⚠️ Không tìm thấy biến toàn cục, đang khởi tạo lại...")
    from llama_index.core import load_index_from_storage, StorageContext
    from pipeline import setup_llm_and_embedding, INDEX_STORAGE

    llm, embed_model = setup_llm_and_embedding()
    storage_context = StorageContext.from_defaults(persist_dir=INDEX_STORAGE)
    vector_index = load_index_from_storage(storage_context)

Settings.llm = llm

# ========================
# 3️⃣ MEMORY CHAT
# ========================
memory = ChatMemoryBuffer.from_defaults(token_limit=1500)
rag_prompt = PromptTemplate(RAG_PROMPT_TEMPLATE)

# ========================
# 4️⃣ CONDITIONAL RAG
# ========================
def should_use_rag(message: str) -> bool:
    """Trigger RAG only if the message likely needs DSM-5 data"""
    keywords = ["DSM-5", "triệu chứng", "rối loạn", "chẩn đoán", "phân loại", "trầm cảm", "lo âu"]
    return any(k.lower() in message.lower() for k in keywords)

# ========================
# 5️⃣ RE-RANKER bằng miniLM / cross-encoder
# ========================
from sentence_transformers import SentenceTransformer, util
import torch

# Load cross-encoder model (miniLM)
reranker_model = SentenceTransformer("cross-encoder/ms-marco-MiniLM-L-6-v2")  # tương đối nhẹ, tốc độ nhanh

def rag_retrieve_and_rerank(message: str, top_k=10, final_top=5) -> str:
    """Retrieve top-k docs, re-rank using cross-encoder, summarize top-n"""
    # 1️⃣ Retrieval
    retrieved_docs = vector_index.similarity_search(message, top_k=top_k)
    if not retrieved_docs:
        return ""

    # 2️⃣ Re-rank using cross-encoder
    doc_texts = [doc.text for doc in retrieved_docs]
    scores = reranker_model.predict(list(zip([message]*len(doc_texts), doc_texts)))
    ranked_docs = [doc for _, doc in sorted(zip(scores, retrieved_docs), key=lambda x: x[0], reverse=True)]

    # 3️⃣ Take top-n documents
    top_docs = ranked_docs[:final_top]

    # 4️⃣ Summarize context to fit token limit
    context_text = " ".join([doc.text for doc in top_docs])
    if len(context_text.split()) > 500:  # tạm thời giới hạn 500 words
        # dùng LLM summarize context
        context_text = llm.invoke(f"Tóm tắt ngắn gọn văn bản sau, giữ thông tin DSM-5 chính: {context_text}")

    return context_text

# ========================
# 6️⃣ MULTI-TURN CHAT + CONDITIONAL RAG
# ========================
def advanced_chat(message, history=[]):
    try:
        if not message.strip():
            return "❌ Vui lòng nhập câu hỏi hoặc chia sẻ cảm xúc."

        # Lấy context nếu cần RAG
        if should_use_rag(message):
            context = rag_retrieve_and_rerank(message)
            if not context:
                context = "⚠️ Không tìm thấy thông tin liên quan trong DSM-5."
        else:
            context = "LLM trả lời trực tiếp, không cần truy vấn tài liệu DSM-5."

        # Compose prompt với history + context
        prompt = RAG_PROMPT_TEMPLATE.format(user_info="", context=context, user_message=message)

        # Truncate memory nếu vượt token limit
        memory_text = memory.get_memory_as_text()
        if len(memory_text.split()) > 1500:
            summary = llm.invoke(f"Tóm tắt ngắn gọn các cuộc trò chuyện trước đây:\n{memory_text}")
            memory.clear()
            memory.update_memory(summary)

        # Generate response
        response = llm.invoke(prompt)
        clean_text = response.strip()
        if not clean_text:
            clean_text = "⚠️ Mình chưa chắc chắn, bạn có thể hỏi lại hoặc thử câu hỏi khác."

        # Lưu vào history
        memory.update_memory(f"user: {message}\nassistant: {clean_text}")

        return clean_text

    except Exception as e:
        print(traceback.format_exc())
        return f"❌ Lỗi khi xử lý: {e}"

# ========================
# 7️⃣ GRADIO INTERFACE
# ========================
title = "🧠 DSM-5 Chatbot Nâng Cao (Conditional RAG + Re-Ranker + Summarize)"
description = (
    "Trợ lý AI chuyên về sức khỏe tâm thần. "
    "LLM trả lời trực tiếp hoặc truy vấn DSM-5 khi cần, Re-Rank + Summarize context."
)

iface = gr.ChatInterface(
    fn=advanced_chat,
    title=title,
    description=description,
    theme="soft",
    examples=[
        ["Tôi cảm thấy rất mệt và không muốn làm gì cả."],
        ["Gần đây tôi ngủ rất ít, bạn nghĩ tôi bị gì không?"],
        ["Tôi thấy bất an và lo lắng về công việc."],
    ]
)

iface.launch(share=True, inbrowser=True)


In [ ]:
# 4️⃣ Hàm xác định khi nào cần truy vấn RAG
# ============================================================
def should_trigger_rag(message: str) -> bool:
    """Phát hiện xem câu hỏi có cần RAG không"""
    trigger_phrases = [
        "phân tích", "triệu chứng", "rối loạn", "dsm", "trầm cảm",
        "lo âu", "stress", "tôi bị", "hướng dẫn", "chuẩn đoán"
    ]
    return any(kw in message.lower() for kw in trigger_phrases)

# ============================================================
# 5️⃣ Hàm re-rank tài liệu (đơn giản)
# ============================================================
def rerank_nodes(query, nodes, top_n=3):
    """Giả lập re-ranker bằng độ dài văn bản"""
    sorted_nodes = sorted(nodes, key=lambda x: -len(str(x.text)))
    return sorted_nodes[:top_n]

# ============================================================
# 6️⃣ Format lại context đưa vào prompt
# ============================================================
def format_context(nodes):
    blocks = [f"[Đoạn {i+1}]\n{n.text.strip()}" for i, n in enumerate(nodes)]
    return "\n\n".join(blocks)

# ============================================================
# 7️⃣ Hàm chat chính (RAG nâng cao)
# ============================================================
def rag_chat(message, history=[]):
    try:
        if not message.strip():
            return "❌ Vui lòng nhập câu hỏi hoặc chia sẻ cảm xúc."

        # (1) Lưu vào bộ nhớ hội thoại
        memory.put_message("user", message)

        # (2) Kiểm tra có cần RAG hay không
        use_rag = should_trigger_rag(message)

        if not use_rag:
            # Nếu không cần RAG → LLM trả lời tự nhiên
            response = llm.complete(
                f"Người dùng nói: {message}\n"
                "→ Hãy phản hồi nhẹ nhàng, đồng cảm, không dùng thuật ngữ chuyên môn."
            )
            return response.text.strip()

        # (3) Tạo retriever và truy vấn vector index
        retriever = vector_index.as_retriever(similarity_top_k=10)
        query_engine = RetrieverQueryEngine.from_args(retriever=retriever)
        retrieved_nodes = query_engine.retrieve(message)

        if not retrieved_nodes:
            return "⚠️ Không tìm thấy thông tin phù hợp từ DSM-5."

        # (4) Re-rank tài liệu
        top_nodes = rerank_nodes(message, retrieved_nodes, top_n=3)

        # (5) Tạo context
        formatted_context = format_context(top_nodes)

        # (6) Ghép prompt mới
        user_info = memory.get_all()
        prompt = f"""
Bạn là chuyên gia tâm lý AI hỗ trợ người dùng dựa trên tiêu chuẩn DSM-5.

Thông tin người dùng (được nhớ từ hội thoại):
{user_info}

Thông tin trích xuất từ DSM-5:
{formatted_context}

Câu hỏi người dùng:
{message}

→ Hãy trả lời ngắn gọn, có dẫn chứng từ DSM-5 khi cần,
và đưa ra lời khuyên nhẹ nhàng, thân thiện, tránh chẩn đoán y khoa chính thức.
        """

        # (7) Sinh phản hồi từ LLM
        response = llm.complete(prompt)
        answer = response.text.strip()

        # (8) Ghi nhớ phản hồi
        memory.put_message("assistant", answer)
        return answer

    except Exception as e:
        import traceback
        print(traceback.format_exc())
        return f"❌ Lỗi khi xử lý: {e}"

# ============================================================
# 8️⃣ Giao diện Gradio
# ============================================================
chatbot_ui = gr.ChatInterface(
    fn=rag_chat,
    title="🧩 DSM-5 AI Psych Assistant (RAG+Memory)",
    description="Trợ lý tâm lý dựa trên DSM-5. Có thể đồng cảm hoặc phân tích triệu chứng theo yêu cầu.",
)
